In [ ]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "")
)
es.ping()
# true if connection is successfull

True

## Prepare the data

In [4]:
# convert the relevant fields to vectors via BERT model
import pandas as pd
df = pd.read_csv("./myntra_products_catalog.csv").loc[:499]
df.head()

/var/folders/q8/w1v60rz107qfsqsys_y9z0gr0000gr/T/ipykernel_86170/1908251434.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [5]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [6]:
df.fillna("None", inplace=True)

## Convert the Relevant field to vector using BERT model

In [10]:
# you can refer https://sbert.net/docs/pretrained_models.html
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

/Users/kartik/Downloads/experiments/semantic-search-deck/search-sbert/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 484kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 23.4MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 205kB/s]
config.json: 100%|██████████| 571/571 [00:00<00:00, 1.78MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:08<00:00, 48.7MB/s] 
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 4.24MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 21.9MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 764kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 475kB/s]


In [11]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))
df.head()

/Users/kartik/Downloads/experiments/semantic-search-deck/search-sbert/.venv/lib/python3.12/site-packages/transformers/models/mpnet/modeling_mpnet.py:1054: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:425.)
  incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645752, -0.0026341616, -0.003588431, 0.0..."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.024660727, -0.02875543, -0.020332465, 0.03..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.046943273, 0.08182784, 0.048335236, -0.000..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.015098726, -0.010285502, 0.00948732, -0.02..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.017746475, 0.0062095057, 0.021813964, 0.02..."


In [12]:
es.ping()

True

## Create new index in elasticsearch

In [13]:
from index_mapping import index_mapping
es.indices.create(index="all_products", mappings=index_mapping)

/var/folders/q8/w1v60rz107qfsqsys_y9z0gr0000gr/T/ipykernel_86170/939210329.py:2: ElasticsearchWarning: Parameter [similarity] has no effect on type [dense_vector] and will be removed in future
  es.indices.create(index="all_products", mappings=index_mapping)
/var/folders/q8/w1v60rz107qfsqsys_y9z0gr0000gr/T/ipykernel_86170/939210329.py:2: ElasticsearchWarning: Parameter [index] has no effect on type [dense_vector] and will be removed in future
  es.indices.create(index="all_products", mappings=index_mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

## Ingest the data into ES

In [15]:
record_list = df.to_dict("records")

In [ ]:
for record in record_list:
    try: 
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)    